# Teleoperation for hand grasping cloth

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*

"""
Control mechanical hand in virtual simulation environment(UE4/Unity/MuJoCo) or in real world scene(PR2/UR5) 
based the application protocol MQTT of IoT, this script works as a client node which connect to the broker 
agent for data processing and hand controller.
"""
import paho.mqtt.client as mqtt
import json
import time
import sys
sys.path.append("/home/jade/DRL/codes/MuJoCo/Hand_Control/drive_hand/")
from joint_position_calculator import get_pose_new4
from data_importer.importer import Anyone


'''
construct the MQTT client for communicating with MQTT broker and realizing data transmission
'''
class MQTTClient:
    def __init__(self, client_id="MQTT_Client", broker_host="192.168.6.131", broker_port=1883, broker_user=None, broker_pass=None, client_keepalive=60):
        # define mqtt parameters
        self.client_id = client_id
        self.broker_host = broker_host
        self.broker_port = broker_port
        self.broker_user = broker_user
        self.broker_pass = broker_pass
        self.client_keepalive = client_keepalive

        # redefine the topic parameters while dealing with topic
        self.topic_sub = "msg_from_server"
        self.qos_sub = 0
        self.msg_sub = None
        self.topic_pub = "msg_to_server"
        self.qos_pub = 0
        self.msg_pub = None

        # overwrite mqtt functions for specified usage
        self.client = mqtt.Client(self.client_id)
        self.client.on_connect = self.on_connect
        self.client.on_disconnect = self.on_disconnect
        self.client.on_subscribe = self.on_subscribe
        self.client.on_unsubscribe = self.on_unsubscribe
        self.client.on_publish = self.on_publish
        self.client.on_message = self.on_message
        self.client.on_log = self.on_log

        # change connection method and loop method
        self.connect_async = self.client.connect_async #asynchronous connect the broker in a non-blocking manner, and the the connection will not complete until loop_start() is called
        self.connect_srv = self.client.connect_srv #connect a broker using an SRV DNS lookup to obtain the broker address
        self.loop = self.client.loop
        self.loop_start = self.client.loop_start
        self.loop_stop = self.client.loop_stop
        self.loop_forever = self.client.loop_forever #blocking network and will not return until the client calls disconnect()


    def connect(self):
        self.client.connect(self.broker_host, self.broker_port, self.client_keepalive)
        print("Connecting to server[%s]......"%self.broker_host)


    def on_connect(self, client, userdata, flags, rc):
        print("Connected to sever[%s] with result code "%self.broker_host + str(rc))


    def disconnect(self):
        self.client.disconnect()
        print("Disconnecting from server[%s]......"%self.broker_host)
    

    def on_disconnect(self, client, userdata, rc):
        print("Disconnected from server[%s] with result code "%self.broker_host + str(rc))
    

    # @staticmethod
    def subscribe(self):
        self.client.subscribe(self.topic_sub, self.qos_sub)
        print("Subscribing topic[%s] from server[%s]......"%(self.topic_sub, self.broker_host))


    def on_subscribe(self, client, userdata, mid, granted_qos):
        print("Subscribed topic[%s] form server[%s]: "%(self.topic_sub, self.broker_host) + str(mid) + " " + str(granted_qos))


    def unsubscribe(self):
        self.client.unsubscribe(self.topic_sub)
        print("Unsubscribing topic[%s] from server[%s]......"%(self.topic_sub, self.broker_host))
    

    def on_unsubscribe(self, client, userdata, mid):
        print("Unsubscribed topic[%s] from server[%s]: "%(self.topic_sub, self.broker_host) + str(mid) + " " + str(self.qos_sub))
    
    # @staticmethod
    def publish(self):
        self.client.publish(self.topic_pub, payload=self.msg_pub, qos=self.qos_pub, retain=False)
        print("Publishing topic[%s] to server[%s]......"%(self.topic_pub, self.broker_host))


    def on_publish(self, client, userdata, mid):
        print("Published topic[%s] to server[%s]: "%(self.topic_pub, self.broker_host) + str(mid))


    def on_message(self, client, userdata, msg):
        try:
            self.msg_sub=json.loads(msg.payload)
            print("Received msg[%s] from server[%s]"%(self.msg_sub, self.broker_host))
        except json.JSONDecodeError as e:
            print("Parsing Msg Error: %s"%e.msg)
    

    def on_log(self, client, userdata, level, string):
        print(string)


"""
Calculate the hand joint pose for teleoperating the mechanical hand in UE4
"""
# ay = Anyone(5)

# idx=20
# ay.draw_point(idx)
# finger_data = ay.data.gtorig
# print(finger_data)


"""
Construct the instance for the MQTT Client and communicate with the broker server for data transmission
"""
# instance the mqtt client and connect to the broker server
mqtt_client = MQTTClient("Jade-PC")
mqtt_client.connect()

# redefine mqtt client topic_pub/topic_sub parameters
mqtt_client.topic_pub = "hand_pose"
mqtt_client.qos_pub = 0
mqtt_client.topic_sub = "ue4_states"
mqtt_client.qos_sub = 0

# loop the subscribe/publish for data transmission
## receive scene data from UE4
# mqtt_client.subscribe()
# while True:
#     mqtt_client.loop()
#     ue4_states = mqtt_client.msg_sub #for other usage
#     pass
# ## send hand pose data to UE4
# mqtt_client.loop_start()
# while True:
#     hand_pose = get_hand_joint_pose(finger_data)
#     mqtt_client.msg_pub = json.dumps(hand_pose)
#     mqtt_client.publish()
#     pass
hand_pose_zero = {'thumb': [0.0, 0.0, 0.0, 0.0], 'index': [0.0, 0.0, 0.0, 0.0], 'middle': [0.0, 0.0, 0.0, 0.0], 'ring': [0.0, 0.0, 0.0, 0.0], 'pinky': [0.0, 0.0, 0.0, 0.0]}
try:
    mqtt_client.subscribe()
#     mqtt_client.loop_start()
    index = 15
    while index <= 1000:
        ay = Anyone(index)
        finger_data = ay.data.gtorig
        hand_pose = get_pose_new4.get_hand_joint_pose(finger_data)
        mqtt_client.loop(1) #timeout(0.001s)：blocking for 1 ms
        hand_pose.update(second=0.9)
        mqtt_client.msg_pub = json.dumps(hand_pose)
        mqtt_client.publish()
#         print(hand_pose)
        
        mqtt_client.loop(1)
        hand_pose_zero.update(second=0.9)
        mqtt_client.msg_pub = json.dumps(hand_pose_zero)
        mqtt_client.publish()
#         ue4_states = mqtt_client.msg_sub #for other usage
        
#         index = index + 1
except:
    print("Components of MPL Hand in MuJoCo scene have collided, so we exclude the data sample and keep going")
#     continue
        
finally:
    mqtt_client.loop_stop()
#     mqtt_client.disconnect()



Sending CONNECT (u0, p0, wr0, wq0, wf0, c1, k60) client_id=b'Jade-PC'
Connecting to server[192.168.6.131]......
Sending SUBSCRIBE (d0, m1) [(b'ue4_states', 0)]
Subscribing topic[ue4_states] from server[192.168.6.131]......
create data
Loading cache data from /home/jade/DRL/codes/MuJoCo/Hand_Control/data_importer/data_importer/cache/MSRA15Importer_P0_None_gt_200_cache.pkl
/home/jade/DRL/codes/MuJoCo/Hand_Control/data_importer/data_importer/cache/MSRA15Importer_P0_None_gt_200_cache.pkl
Shuffling
you now choosing seq 0 15
Received CONNACK (0, 0)
Connected to sever[192.168.6.131] with result code 0
Sending PUBLISH (d0, q0, r0, m2), 'b'hand_pose'', ... (490 bytes)
Published topic[hand_pose] to server[192.168.6.131]: 2
Publishing topic[hand_pose] to server[192.168.6.131]......
{'thumb': [1.6237809091855413, 0.0002921968397331452, 0.5428007605563286, 0.8444707561183966], 'index': [-0.18085300513728628, 1.3110153281993249, 1.6479933741320232, 0.7473831312155131], 'middle': [-0.0740849989469047

Shuffling
you now choosing seq 0 15
Sending PUBLISH (d0, q0, r0, m16), 'b'hand_pose'', ... (490 bytes)
Published topic[hand_pose] to server[192.168.6.131]: 16
Publishing topic[hand_pose] to server[192.168.6.131]......
{'thumb': [1.6237809091855413, 0.0002921968397331452, 0.5428007605563286, 0.8444707561183966], 'index': [-0.18085300513728628, 1.3110153281993249, 1.6479933741320232, 0.7473831312155131], 'middle': [-0.07408499894690473, 0.15498690718561417, 0.08197957684764551, 0.10360335098124283], 'ring': [0.07041798059385243, 0.02940668423868865, 0.04638966988253528, 0.05929293587474276], 'pinky': [0.48259104158246036, -0.010551544827392203, 0.10347492764892102, 0.07752210992972099], 'second': 0.9}
Sending PUBLISH (d0, q0, r0, m17), 'b'hand_pose'', ... (170 bytes)
Published topic[hand_pose] to server[192.168.6.131]: 17
Publishing topic[hand_pose] to server[192.168.6.131]......
create data
Loading cache data from /home/jade/DRL/codes/MuJoCo/Hand_Control/data_importer/data_importer/cach

Shuffling
you now choosing seq 0 15
Sending PUBLISH (d0, q0, r0, m32), 'b'hand_pose'', ... (490 bytes)
Published topic[hand_pose] to server[192.168.6.131]: 32
Publishing topic[hand_pose] to server[192.168.6.131]......
{'thumb': [1.6237809091855413, 0.0002921968397331452, 0.5428007605563286, 0.8444707561183966], 'index': [-0.18085300513728628, 1.3110153281993249, 1.6479933741320232, 0.7473831312155131], 'middle': [-0.07408499894690473, 0.15498690718561417, 0.08197957684764551, 0.10360335098124283], 'ring': [0.07041798059385243, 0.02940668423868865, 0.04638966988253528, 0.05929293587474276], 'pinky': [0.48259104158246036, -0.010551544827392203, 0.10347492764892102, 0.07752210992972099], 'second': 0.9}
Sending PUBLISH (d0, q0, r0, m33), 'b'hand_pose'', ... (170 bytes)
Published topic[hand_pose] to server[192.168.6.131]: 33
Publishing topic[hand_pose] to server[192.168.6.131]......
create data
Loading cache data from /home/jade/DRL/codes/MuJoCo/Hand_Control/data_importer/data_importer/cach

Shuffling
you now choosing seq 0 15
Sending PUBLISH (d0, q0, r0, m48), 'b'hand_pose'', ... (490 bytes)
Published topic[hand_pose] to server[192.168.6.131]: 48
Publishing topic[hand_pose] to server[192.168.6.131]......
{'thumb': [1.6237809091855413, 0.0002921968397331452, 0.5428007605563286, 0.8444707561183966], 'index': [-0.18085300513728628, 1.3110153281993249, 1.6479933741320232, 0.7473831312155131], 'middle': [-0.07408499894690473, 0.15498690718561417, 0.08197957684764551, 0.10360335098124283], 'ring': [0.07041798059385243, 0.02940668423868865, 0.04638966988253528, 0.05929293587474276], 'pinky': [0.48259104158246036, -0.010551544827392203, 0.10347492764892102, 0.07752210992972099], 'second': 0.9}
Sending PUBLISH (d0, q0, r0, m49), 'b'hand_pose'', ... (170 bytes)
Published topic[hand_pose] to server[192.168.6.131]: 49
Publishing topic[hand_pose] to server[192.168.6.131]......
create data
Loading cache data from /home/jade/DRL/codes/MuJoCo/Hand_Control/data_importer/data_importer/cach

Shuffling
you now choosing seq 0 15
Sending PUBLISH (d0, q0, r0, m64), 'b'hand_pose'', ... (490 bytes)
Published topic[hand_pose] to server[192.168.6.131]: 64
Publishing topic[hand_pose] to server[192.168.6.131]......
{'thumb': [1.6237809091855413, 0.0002921968397331452, 0.5428007605563286, 0.8444707561183966], 'index': [-0.18085300513728628, 1.3110153281993249, 1.6479933741320232, 0.7473831312155131], 'middle': [-0.07408499894690473, 0.15498690718561417, 0.08197957684764551, 0.10360335098124283], 'ring': [0.07041798059385243, 0.02940668423868865, 0.04638966988253528, 0.05929293587474276], 'pinky': [0.48259104158246036, -0.010551544827392203, 0.10347492764892102, 0.07752210992972099], 'second': 0.9}
Sending PUBLISH (d0, q0, r0, m65), 'b'hand_pose'', ... (170 bytes)
Published topic[hand_pose] to server[192.168.6.131]: 65
Publishing topic[hand_pose] to server[192.168.6.131]......
create data
Loading cache data from /home/jade/DRL/codes/MuJoCo/Hand_Control/data_importer/data_importer/cach

...

Shuffling
you now choosing seq 0 15
Sending PUBLISH (d0, q0, r0, m480), 'b'hand_pose'', ... (490 bytes)
Published topic[hand_pose] to server[192.168.6.131]: 480
Publishing topic[hand_pose] to server[192.168.6.131]......
{'thumb': [1.6237809091855413, 0.0002921968397331452, 0.5428007605563286, 0.8444707561183966], 'index': [-0.18085300513728628, 1.3110153281993249, 1.6479933741320232, 0.7473831312155131], 'middle': [-0.07408499894690473, 0.15498690718561417, 0.08197957684764551, 0.10360335098124283], 'ring': [0.07041798059385243, 0.02940668423868865, 0.04638966988253528, 0.05929293587474276], 'pinky': [0.48259104158246036, -0.010551544827392203, 0.10347492764892102, 0.07752210992972099], 'second': 0.9}
Sending PUBLISH (d0, q0, r0, m481), 'b'hand_pose'', ... (170 bytes)
Published topic[hand_pose] to server[192.168.6.131]: 481
Publishing topic[hand_pose] to server[192.168.6.131]......
create data
Loading cache data from /home/jade/DRL/codes/MuJoCo/Hand_Control/data_importer/data_importer/

Shuffling
you now choosing seq 0 15
Sending PUBLISH (d0, q0, r0, m496), 'b'hand_pose'', ... (490 bytes)
Published topic[hand_pose] to server[192.168.6.131]: 496
Publishing topic[hand_pose] to server[192.168.6.131]......
{'thumb': [1.6237809091855413, 0.0002921968397331452, 0.5428007605563286, 0.8444707561183966], 'index': [-0.18085300513728628, 1.3110153281993249, 1.6479933741320232, 0.7473831312155131], 'middle': [-0.07408499894690473, 0.15498690718561417, 0.08197957684764551, 0.10360335098124283], 'ring': [0.07041798059385243, 0.02940668423868865, 0.04638966988253528, 0.05929293587474276], 'pinky': [0.48259104158246036, -0.010551544827392203, 0.10347492764892102, 0.07752210992972099], 'second': 0.9}
Sending PUBLISH (d0, q0, r0, m497), 'b'hand_pose'', ... (170 bytes)
Published topic[hand_pose] to server[192.168.6.131]: 497
Publishing topic[hand_pose] to server[192.168.6.131]......
create data
Loading cache data from /home/jade/DRL/codes/MuJoCo/Hand_Control/data_importer/data_importer/

Shuffling
you now choosing seq 0 15
Sending PUBLISH (d0, q0, r0, m512), 'b'hand_pose'', ... (490 bytes)
Published topic[hand_pose] to server[192.168.6.131]: 512
Publishing topic[hand_pose] to server[192.168.6.131]......
{'thumb': [1.6237809091855413, 0.0002921968397331452, 0.5428007605563286, 0.8444707561183966], 'index': [-0.18085300513728628, 1.3110153281993249, 1.6479933741320232, 0.7473831312155131], 'middle': [-0.07408499894690473, 0.15498690718561417, 0.08197957684764551, 0.10360335098124283], 'ring': [0.07041798059385243, 0.02940668423868865, 0.04638966988253528, 0.05929293587474276], 'pinky': [0.48259104158246036, -0.010551544827392203, 0.10347492764892102, 0.07752210992972099], 'second': 0.9}
Sending PUBLISH (d0, q0, r0, m513), 'b'hand_pose'', ... (170 bytes)
Published topic[hand_pose] to server[192.168.6.131]: 513
Publishing topic[hand_pose] to server[192.168.6.131]......
create data
Loading cache data from /home/jade/DRL/codes/MuJoCo/Hand_Control/data_importer/data_importer/

Shuffling
you now choosing seq 0 15
Sending PUBLISH (d0, q0, r0, m528), 'b'hand_pose'', ... (490 bytes)
Published topic[hand_pose] to server[192.168.6.131]: 528
Publishing topic[hand_pose] to server[192.168.6.131]......
{'thumb': [1.6237809091855413, 0.0002921968397331452, 0.5428007605563286, 0.8444707561183966], 'index': [-0.18085300513728628, 1.3110153281993249, 1.6479933741320232, 0.7473831312155131], 'middle': [-0.07408499894690473, 0.15498690718561417, 0.08197957684764551, 0.10360335098124283], 'ring': [0.07041798059385243, 0.02940668423868865, 0.04638966988253528, 0.05929293587474276], 'pinky': [0.48259104158246036, -0.010551544827392203, 0.10347492764892102, 0.07752210992972099], 'second': 0.9}
Sending PUBLISH (d0, q0, r0, m529), 'b'hand_pose'', ... (170 bytes)
Published topic[hand_pose] to server[192.168.6.131]: 529
Publishing topic[hand_pose] to server[192.168.6.131]......
create data
Loading cache data from /home/jade/DRL/codes/MuJoCo/Hand_Control/data_importer/data_importer/

Shuffling
you now choosing seq 0 15
Sending PUBLISH (d0, q0, r0, m544), 'b'hand_pose'', ... (490 bytes)
Published topic[hand_pose] to server[192.168.6.131]: 544
Publishing topic[hand_pose] to server[192.168.6.131]......
{'thumb': [1.6237809091855413, 0.0002921968397331452, 0.5428007605563286, 0.8444707561183966], 'index': [-0.18085300513728628, 1.3110153281993249, 1.6479933741320232, 0.7473831312155131], 'middle': [-0.07408499894690473, 0.15498690718561417, 0.08197957684764551, 0.10360335098124283], 'ring': [0.07041798059385243, 0.02940668423868865, 0.04638966988253528, 0.05929293587474276], 'pinky': [0.48259104158246036, -0.010551544827392203, 0.10347492764892102, 0.07752210992972099], 'second': 0.9}
Sending PUBLISH (d0, q0, r0, m545), 'b'hand_pose'', ... (170 bytes)
Published topic[hand_pose] to server[192.168.6.131]: 545
Publishing topic[hand_pose] to server[192.168.6.131]......
create data
Loading cache data from /home/jade/DRL/codes/MuJoCo/Hand_Control/data_importer/data_importer/

# Appendix

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*

"""
Control mechanical hand in virtual simulation environment(UE4/Unity/MuJoCo) or in real world scene(PR2/UR5) 
based the application protocol MQTT of IoT, this script works as a client node which connect to the broker 
agent for data processing and hand controller.
"""
import paho.mqtt.client as mqtt
import json
import time
import sys
# import pyquaternion as pq
sys.path.append("D:\Tempfiles\PycharmProjects\DRL\codes\MuJoCo\Hand_Control\drive_hand")
# from joint_position_calculator.get_angle_new6 import *


'''
construct the MQTT client for communicating with MQTT broker and realizing data transmission
'''
class MQTTClient:
    def __init__(self, client_id="MQTT_Client", broker_host="192.168.6.131", broker_port=1883, broker_user=None, broker_pass=None, client_keepalive=60):
        # define mqtt parameters
        self.client_id = client_id
        self.broker_host = broker_host
        self.broker_port = broker_port
        self.broker_user = broker_user
        self.broker_pass = broker_pass
        self.client_keepalive = client_keepalive

        # redefine the topic parameters while dealing with topic
        self.topic_sub = "msg_from_server"
        self.qos_sub = 0
        self.msg_sub = None
        self.topic_pub = "msg_to_server"
        self.qos_pub = 0
        self.msg_pub = None

        # overwrite mqtt functions for specified usage
        self.client = mqtt.Client(self.client_id)
        self.client.on_connect = self.on_connect
        self.client.on_disconnect = self.on_disconnect
        self.client.on_subscribe = self.on_subscribe
        self.client.on_unsubscribe = self.on_unsubscribe
        self.client.on_publish = self.on_publish
        self.client.on_message = self.on_message
        self.client.on_log = self.on_log

        # change connection method and loop method
        self.connect_async = self.client.connect_async #asynchronous connect the broker in a non-blocking manner, and the the connection will not complete until loop_start() is called
        self.connect_srv = self.client.connect_srv #connect a broker using an SRV DNS lookup to obtain the broker address
        self.loop = self.client.loop
        self.loop_start = self.client.loop_start
        self.loop_stop = self.client.loop_stop
        self.loop_forever = self.client.loop_forever #blocking network and will not return until the client calls disconnect()


    def connect(self):
        self.client.connect(self.broker_host, self.broker_port, self.client_keepalive)
        print("Connecting to server[%s]......"%self.broker_host)


    def on_connect(self, client, userdata, flags, rc):
        print("Connected to sever[%s] with result code "%self.broker_host + str(rc))


    def disconnect(self):
        self.client.disconnect()
        print("Disconnecting from server[%s]......"%self.broker_host)
    

    def on_disconnect(self, client, userdata, rc):
        print("Disconnected from server[%s] with result code "%self.broker_host + str(rc))
    

    # @staticmethod
    def subscribe(self):
        self.client.subscribe(self.topic_sub, self.qos_sub)
        print("Subscribing topic[%s] from server[%s]......"%(self.topic_sub, self.broker_host))


    def on_subscribe(self, client, userdata, mid, granted_qos):
        print("Subscribed topic[%s] form server[%s]: "%(self.topic_sub, self.broker_host) + str(mid) + " " + str(granted_qos))


    def unsubscribe(self):
        self.client.unsubscribe(self.topic_sub)
        print("Unsubscribing topic[%s] from server[%s]......"%(self.topic_sub, self.broker_host))
    

    def on_unsubscribe(self, client, userdata, mid):
        print("Unsubscribed topic[%s] from server[%s]: "%(self.topic_sub, self.broker_host) + str(mid) + " " + str(self.qos_sub))
    
    # @staticmethod
    def publish(self):
        self.client.publish(self.topic_pub, payload=self.msg_pub, qos=self.qos_pub, retain=False)
        print("Publishing topic[%s] to server[%s]......"%(self.topic_pub, self.broker_host))


    def on_publish(self, client, userdata, mid):
        print("Published topic[%s] to server[%s]: "%(self.topic_pub, self.broker_host) + str(mid))


    def on_message(self, client, userdata, msg):
        try:
            self.msg_sub=json.loads(msg.payload)
            print("Received msg[%s] from server[%s]"%(self.msg_sub, self.broker_host))
        except json.JSONDecodeError as e:
            print("Parsing Msg Error: %s"%e.msg)
        
        # return self.msg_sub
    

    def on_log(self, client, userdata, level, string):
        print(string)
        
    
"""
Construct the instance for the MQTT Client and communicate with the broker server for data transmission
"""
# instance the mqtt client and connect to the broker server
mqtt_client = MQTTClient("Jade-PC")
mqtt_client.connect()

# redefine mqtt client topic_pub/topic_sub parameters
mqtt_client.topic_pub = "hand_pose"
mqtt_client.qos_pub = 0
mqtt_client.topic_sub = "ue4_states"
mqtt_client.qos_sub = 0

# loop the subscribe/publish for data transmission
## receive scene data from UE4
mqtt_client.subscribe()
while True:
    mqtt_client.loop()
    ue4_states = mqtt_client.msg_sub #for other usage
    pass
## send hand pose data to UE4
mqtt_client.loop_start()
while True:
    finger_data = 0.0
    hand_pose = get_hand_joint_pose(finger_data)
    mqtt_client.msg_pub = json.dumps(hand_pose)
    mqtt_client.publish()
    pass



Sending CONNECT (u0, p0, wr0, wq0, wf0, c1, k60) client_id=b'Jade-PC'
Connecting to server[192.168.6.131]......
Sending SUBSCRIBE (d0, m1) [(b'ue4_states', 0)]
Subscribing topic[ue4_states] from server[192.168.6.131]......
Received CONNACK (0, 0)
Connected to sever[192.168.6.131] with result code 0
Received SUBACK
Subscribed topic[ue4_states] form server[192.168.6.131]: 1 (0,)
